In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from constraint import Problem

## Question 1

In [2]:
class Node:
    def __init__(self, value=None, children=None):
        self.value = value  
        self.children = children if children else []  # Child nodes (empty for leaf nodes)

# Alpha-Beta Pruning function
def alpha_beta(node, alpha, beta, maximizing_player, pruned_nodes, depth=0):
    indent = '  ' * depth
    if not node.children:
        return node.value
    
    pruned = False  
    
    if maximizing_player:
        max_eval = float('-inf')
        for child in node.children:
            eval = alpha_beta(child, alpha, beta, False, pruned_nodes, depth + 1)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            
            if beta <= alpha:  # Pruning condition
                if not pruned:
                    pruned_nodes.append(child)  # Append pruned child node
                    print(f"Pruning at Max node with alpha={alpha}, beta={beta}")
                    pruned = True  # Set flag to avoid duplicate printing
                break  # Beta cutoff (prune the remaining children)
        return max_eval

    else:
        # Minimizing player
        min_eval = float('inf')
        for child in node.children:
            eval = alpha_beta(child, alpha, beta, True, pruned_nodes, depth + 1)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            
            if beta <= alpha:  # Pruning condition
                if not pruned:
                    pruned_nodes.append(child)  # Append pruned child node
                    print(f"Pruning at Min node with alpha={alpha}, beta={beta}")
                    pruned = True  # Set flag to avoid duplicate printing
                break  # Alpha cutoff (prune the remaining children)
        return min_eval

# Tree structure 

#          root node
#          /       \
#  leaf1=5         min_node_1
#                   /       \
#         max_node_1          max_node_2
#        /         \         /          \
# leaf2=1      min_node_2 leaf3=5      min_node_3
#              /        \              /         \
#       leaf4=4       leaf5=2     leaf6=4     leaf7=3

# Nodes defined
leaf1 = Node(value=5)
leaf2 = Node(value=1)
leaf5 = Node(value=2)
leaf4 = Node(value=4)
leaf3 = Node(value=5)
leaf6 = Node(value=4)
leaf7 = Node(value=3)
min_node3 = Node(children=[leaf6, leaf7])
min_node2 = Node(children=[leaf5, leaf4]) 
max_node2 = Node(children=[leaf3, min_node3])
max_node1 = Node(children=[leaf2, min_node2]) 
min_node1 = Node(children=[max_node1, max_node2])
root = Node(children=[leaf1, min_node1])

pruned_nodes = []

# Run Alpha-Beta Pruning
result = alpha_beta(root, float('-inf'), float('inf'), True, pruned_nodes)

# Display results
print(f"\nResult of Alpha-Beta Pruning (Min-Max value of root): {result}")


Pruning at Min node with alpha=5, beta=2
Pruning at Min node with alpha=5, beta=2

Result of Alpha-Beta Pruning (Min-Max value of root): 5


## Question 2

In [3]:
# Define constants 
DOCTORS = ['Doctor 1', 'Doctor 2']
SERVICES = {
    'Routine_Checkup': 7,  # 9am - 4pm
    'Blood_Test': 4,       # 9am - 1pm
    'Surgery': 5           # 10am - 3pm
}

# Define the weekly schedule 
WEEKDAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
SERVICE_DAYS = {
    'Routine_Checkup': ['Monday', 'Tuesday', 'Wednesday', 'Friday'],
    'Blood_Test': ['Wednesday'],
    'Surgery': ['Thursday']
}

# Define number of weeks 
NUM_WEEKS = 2  

# Initialize 
problem = Problem()

# Add variables for each service in each week, with domains of the doctors
for week in range(1, NUM_WEEKS + 1):
    for service in SERVICES.keys():
        for day in SERVICE_DAYS[service]:
            service_name = f"{service}_{day}_Week{week}"
            problem.addVariable(service_name, DOCTORS)

# Constraint 1: Balance the total hours worked for both doctors across all weeks
def balance_workload(*assigned_doctors):
    doctor_hours = {'Doctor 1': 0, 'Doctor 2': 0}
    idx = 0
    for service in SERVICES.keys():
        for day in SERVICE_DAYS[service]:
            for week in range(1, NUM_WEEKS + 1):
                service_name = f"{service}_{day}_Week{week}"
                doctor_hours[assigned_doctors[idx]] += SERVICES[service]
                idx += 1
    # Ensure both doctors have equal workload
    return doctor_hours['Doctor 1'] == doctor_hours['Doctor 2']


problem.addConstraint(balance_workload, [f"{service}_{day}_Week{week}" for service in SERVICES.keys() for day in SERVICE_DAYS[service] for week in range(1, NUM_WEEKS + 1)])

# Get solutions to the CSP
solutions = problem.getSolutions()

# Display solutions 
if solutions:
    solution = solutions[0]  # first solution
    print("Service Schedule")

    for service, days in SERVICE_DAYS.items():
        for day in days:
            for week in range(1, NUM_WEEKS + 1):
                service_name = f"{service}_{day}_Week{week}"
                if service_name in solution:
                    doctor = solution[service_name]
                    # Print the doctor assignments for each service, week, and day
                    if doctor == "Doctor 1":
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr1:\t{SERVICES[service]}")
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr2:\t0")
                    elif doctor == "Doctor 2":
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr1:\t0")
                        print(f"Week{week}_{day}_{service.replace('_', ' ')}_Dr2:\t{SERVICES[service]}")
else:
    print("No solution found")

Service Schedule
Week1_Monday_Routine Checkup_Dr1:	0
Week1_Monday_Routine Checkup_Dr2:	7
Week2_Monday_Routine Checkup_Dr1:	0
Week2_Monday_Routine Checkup_Dr2:	7
Week1_Tuesday_Routine Checkup_Dr1:	7
Week1_Tuesday_Routine Checkup_Dr2:	0
Week2_Tuesday_Routine Checkup_Dr1:	7
Week2_Tuesday_Routine Checkup_Dr2:	0
Week1_Wednesday_Routine Checkup_Dr1:	7
Week1_Wednesday_Routine Checkup_Dr2:	0
Week2_Wednesday_Routine Checkup_Dr1:	7
Week2_Wednesday_Routine Checkup_Dr2:	0
Week1_Friday_Routine Checkup_Dr1:	0
Week1_Friday_Routine Checkup_Dr2:	7
Week2_Friday_Routine Checkup_Dr1:	0
Week2_Friday_Routine Checkup_Dr2:	7
Week1_Wednesday_Blood Test_Dr1:	0
Week1_Wednesday_Blood Test_Dr2:	4
Week2_Wednesday_Blood Test_Dr1:	4
Week2_Wednesday_Blood Test_Dr2:	0
Week1_Thursday_Surgery_Dr1:	0
Week1_Thursday_Surgery_Dr2:	5
Week2_Thursday_Surgery_Dr1:	5
Week2_Thursday_Surgery_Dr2:	0


In [4]:
number_of_solutions = len(solutions)
print(f"Number of possible solutions: {number_of_solutions}")

Number of possible solutions: 280
